In [148]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
import pandas as pd
from tqdm import tqdm

In [106]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2} # prevents loading images and increases the loading speed
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome(options=chrome_options)

In [4]:
wd.get('https://akharinkhabar.ir/')

In [29]:
# getting categories' names from navbar
categories = []
nav_bar = wd.find_elements(By.XPATH, '/html/body/header/div/nav/ul/li/a')
for attr in nav_bar:
    categories.append(attr.get_attribute(
        'href').replace('https://akharinkhabar.ir/', ''))

# some categories are useless, we don't need them
categories = categories[1:14]

In [ ]:
for cat in categories:
    print(f'------------------ {cat} ---------------------')
    wd.get(f'https://akharinkhabar.ir/{cat}')
    for c in tqdm(range(1000)):
        # clicks on load more for 1000 times
        try:
            loadMoreButton = wd.find_element(
                By.XPATH, './/*[@id="metroList_load"]')
            time.sleep(2)
            loadMoreButton.click()
            time.sleep(2)
        except Exception as e:
            print(e)
            break
    
    # gets the links and save them to a dataframe(category based)
    news_links = []
    a_tags = wd.find_elements(
        By.XPATH, './/*[@id="module-list"]/div[1]/ul/li/a')
    for url in a_tags:
        news_links.append(url.get_attribute('href'))
        time.sleep(1)
    df = pd.DataFrame({'links': news_links})
    df.to_csv(f'{cat}.csv')
    print(f'--------- {cat} done and saved-----------')

    time.sleep(2)

In [ ]:
for cat in categories:
    print('\n', f'crawling data from {cat} category')
    df = pd.read_csv(f'{cat}.csv')
    for i in tqdm(df.index):
        wd.get(df['links'][i])
        main_box = wd.find_elements(By.CLASS_NAME, 'story-main-box')

        for section in main_box:
            title = section.find_element(
                By.XPATH, './/*[@id="module-position-show-title"]').text
            datetime = section.find_element(
                By.CSS_SELECTOR, '#module-position-lable > span.asset-metabar-time.asset-metabar-item.nobyline.font_homa_10').text
            source = section.find_element(
                By.CSS_SELECTOR, '#module-position-lable > span.asset-metabar-owner.asset-metabar-item.font_homa_10 > a > span').text
            text = section.find_element(
                By.CSS_SELECTOR, '#view-module > div.story-main-box > div > p:nth-child(3)').text
        df.at[i, 'title'] = title
        df.at[i, 'datetime'] = datetime
        df.at[i, 'source'] = source
        df.at[i, 'text'] = text

    df.to_csv(f'{cat}_all.csv')
    categories.remove(cat)